In [1]:
import numpy as np
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
from torchsummary import summary
from torchvision import transforms

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [2]:
class hw8_conv_network(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.main = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3, 3)),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d((2, 2)),
            torch.nn.Flatten(),
            torch.nn.Linear(313632, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=64, out_features=1),
            # torch.nn.Sigmoid()
        )

    def forward(self, x):
        out = self.main(x)
        return out

# Question 1

In [3]:
model = hw8_conv_network()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
summary(model, (3, 200, 200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 198, 198]             896
              ReLU-2         [-1, 32, 198, 198]               0
         MaxPool2d-3           [-1, 32, 99, 99]               0
           Flatten-4               [-1, 313632]               0
            Linear-5                   [-1, 64]      20,072,512
              ReLU-6                   [-1, 64]               0
            Linear-7                    [-1, 1]              65
Total params: 20,073,473
Trainable params: 20,073,473
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 23.93
Params size (MB): 76.57
Estimated Total Size (MB): 100.96
----------------------------------------------------------------


In [4]:
original_transforms = [
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
]  # ImageNet normalization
train_transforms = transforms.Compose(original_transforms)

In [5]:
train_dataset = torchvision.datasets.ImageFolder(
    root="./data/train", transform=train_transforms
)
validation_dataset = torchvision.datasets.ImageFolder(
    root="./data/test", transform=train_transforms
)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=20, shuffle=False)

# Question 2

In [6]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)
criterion = torch.nn.BCEWithLogitsLoss()

# Question 3

In [7]:
num_epochs = 10
history = {"acc": [], "loss": [], "val_acc": [], "val_loss": []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(
            1
        )  # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history["loss"].append(epoch_loss)
    history["acc"].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history["val_loss"].append(val_epoch_loss)
    history["val_acc"].append(val_epoch_acc)

    print(
        f"Epoch {epoch + 1}/{num_epochs}, "
        f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
        f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}"
    )

Epoch 1/10, Loss: 0.6665, Acc: 0.6112, Val Loss: 0.6511, Val Acc: 0.6617
Epoch 2/10, Loss: 0.5702, Acc: 0.6787, Val Loss: 0.6332, Val Acc: 0.6318
Epoch 3/10, Loss: 0.5207, Acc: 0.7350, Val Loss: 0.6143, Val Acc: 0.6766
Epoch 4/10, Loss: 0.4773, Acc: 0.7600, Val Loss: 0.6049, Val Acc: 0.6617
Epoch 5/10, Loss: 0.4606, Acc: 0.7550, Val Loss: 0.7307, Val Acc: 0.5672
Epoch 6/10, Loss: 0.3954, Acc: 0.8275, Val Loss: 0.6412, Val Acc: 0.6866
Epoch 7/10, Loss: 0.2844, Acc: 0.8838, Val Loss: 0.8307, Val Acc: 0.6816
Epoch 8/10, Loss: 0.2885, Acc: 0.8788, Val Loss: 0.7052, Val Acc: 0.7114
Epoch 9/10, Loss: 0.1882, Acc: 0.9313, Val Loss: 0.9275, Val Acc: 0.6866
Epoch 10/10, Loss: 0.2585, Acc: 0.8912, Val Loss: 0.8158, Val Acc: 0.6915


In [8]:
training_accuracy = np.array(history["acc"])
training_loss = np.array(history["loss"])
validation_accuracy = np.array(history["val_acc"])
validation_loss = np.array(history["val_loss"])

In [9]:
np.median(training_accuracy)

np.float64(0.79375)

# Question 4

In [10]:
np.std(training_loss)

np.float64(0.14617715818261098)

In [11]:
new_transforms = [
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
]

In [12]:
train_dataset.transform = transforms.Compose([*original_transforms, *new_transforms])

In [13]:
train_dataset.transform

Compose(
    Resize(size=(200, 200), interpolation=bilinear, max_size=None, antialias=True)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    RandomRotation(degrees=[-50.0, 50.0], interpolation=nearest, expand=False, fill=0)
    RandomResizedCrop(size=(200, 200), scale=(0.9, 1.0), ratio=(0.9, 1.1), interpolation=bilinear, antialias=True)
    RandomHorizontalFlip(p=0.5)
)

In [14]:
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)

In [ ]:
num_epochs = 10
# history = {"acc": [], "loss": [], "val_acc": [], "val_loss": []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(
            1
        )  # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history["loss"].append(epoch_loss)
    history["acc"].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history["val_loss"].append(val_epoch_loss)
    history["val_acc"].append(val_epoch_acc)

    print(
        f"Epoch {epoch + 1}/{num_epochs}, "
        f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
        f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}"
    )

Epoch 1/10, Loss: 0.6409, Acc: 0.6587, Val Loss: 0.6042, Val Acc: 0.6866
Epoch 2/10, Loss: 0.5797, Acc: 0.7037, Val Loss: 0.5816, Val Acc: 0.7015
Epoch 3/10, Loss: 0.5634, Acc: 0.6875, Val Loss: 0.5569, Val Acc: 0.7313
Epoch 4/10, Loss: 0.5390, Acc: 0.7300, Val Loss: 0.5672, Val Acc: 0.7114
Epoch 5/10, Loss: 0.5352, Acc: 0.7137, Val Loss: 0.5731, Val Acc: 0.7214
Epoch 6/10, Loss: 0.5121, Acc: 0.7288, Val Loss: 0.5452, Val Acc: 0.7363
Epoch 7/10, Loss: 0.5079, Acc: 0.7562, Val Loss: 0.5856, Val Acc: 0.7065
Epoch 8/10, Loss: 0.5416, Acc: 0.7050, Val Loss: 0.5382, Val Acc: 0.7313
Epoch 9/10, Loss: 0.4944, Acc: 0.7338, Val Loss: 0.5626, Val Acc: 0.7363
Epoch 10/10, Loss: 0.5071, Acc: 0.7475, Val Loss: 0.5950, Val Acc: 0.7463


In [16]:
training_accuracy = np.array(history["acc"])
training_loss = np.array(history["loss"])
validation_accuracy = np.array(history["val_acc"])
validation_loss = np.array(history["val_loss"])

# Question 5

In [19]:
np.mean(validation_loss[10:])

np.float64(0.5709663242917156)

# Question 6

In [20]:
np.mean(validation_accuracy[-6:])

np.float64(0.7296849087893865)